In [43]:
import imp
imp.reload(get_outliers)

<module 'get_outliers' from '/Users/diogomesquita/Documents/nyu/1y/2s/1004/big_data_project/get_outliers.py'>

### Clean data and find NULLs

In [18]:
import os

data_dir = 'GROUP7_rid'
file_prefix = os.path.join(data_dir, 'bss9-579f')
src_dir = '.'

In [19]:
%%bash -s "$file_prefix" "$src_dir"
source activate py3.6
file_prefix=$1
src_dir=$2

python $src_dir/clean_data.py $file_prefix.tsv

2018-05-03 10:29:48 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-05-03 10:29:51 WARN  DataSource:66 - Multiple sources found for csv (org.apache.spark.sql.execution.datasources.csv.CSVFileFormat, com.databricks.spark.csv.DefaultSource15), defaulting to the internal datasource (org.apache.spark.sql.execution.datasources.csv.CSVFileFormat).
2018-05-03 10:29:51 WARN  DataSource:66 - Multiple sources found for csv (org.apache.spark.sql.execution.datasources.csv.CSVFileFormat, com.databricks.spark.csv.DefaultSource15), defaulting to the internal datasource (org.apache.spark.sql.execution.datasources.csv.CSVFileFormat).
2018-05-03 10:29:56 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
2018-05-03 10:30:04 WARN  DataSource:66 - Multiple sources found for csv (org.apache.spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Find outliers

In [45]:
%%bash -s "$file_prefix" "$src_dir"
source activate py3.6
file_prefix=$1
src_dir=$2

python $src_dir/get_outliers.py ${file_prefix}_clean.tsv

2018-05-03 11:18:29 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-05-03 11:18:30 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2018-05-03 11:18:31 WARN  DataSource:66 - Multiple sources found for csv (org.apache.spark.sql.execution.datasources.csv.CSVFileFormat, com.databricks.spark.csv.DefaultSource15), defaulting to the internal datasource (org.apache.spark.sql.execution.datasources.csv.CSVFileFormat).
2018-05-03 11:18:31 WARN  DataSource:66 - Multiple sources found for csv (org.apache.spark.sql.execution.datasources.csv.CSVFileFormat, com.databricks.spark.csv.DefaultSource15), defaulting to the internal datasource (org.apache.spark.sql.execution.datasources.csv.CSVFileFormat).
2018-05-03 11:18:36 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [31]:
import utils
import get_outliers
import numpy as np

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel

spark = SparkSession \
            .builder \
            .appName("Nulls and Outliers Detection") \
            .getOrCreate()

In [25]:
df = spark.read.csv("{}_clean.tsv".format(file_prefix), header = True,inferSchema = True, sep='\t')

In [40]:
df_count = df.count()
kmeans = {}
for col, dtype in df.dtypes:
    print("### Col: {} | dtype: {} ###".format(col, dtype))
    if col == 'rid':
        continue
    if 'string' not in dtype:
        kmeans[col] = get_outliers.kmeans_outliers(df, col)
        print("\tKmeans:\n\t\tcounts: {}\n\t\tcounts % = {:.2f}%".format(kmeans[col].count(), 100*(kmeans[col].count()/df_count)))


### Col: comparable_rental_2_address | dtype: string ###
### Col: brooklyn_condominiums_comparable_properties_address | dtype: string ###
### Col: comparable_rental_2_gross_sqft | dtype: int ###
	Kmeans:
		counts: 6
		counts % = 1.40%
### Col: comparable_rental_3_estimated_gross_income | dtype: double ###
	Kmeans:
		counts: 2
		counts % = 0.47%
### Col: brooklyn_condominiums_comparable_properties_estimated_expense | dtype: int ###
	Kmeans:
		counts: 5
		counts % = 1.17%
### Col: comparable_rental_2_building_classification | dtype: string ###
### Col: brooklyn_condominiums_comparable_properties_total_units | dtype: int ###
	Kmeans:
		counts: 1
		counts % = 0.23%
### Col: brooklyn_condominiums_comparable_properties_expense_per_sqft | dtype: double ###
	Kmeans:
		counts: 2
		counts % = 0.47%
### Col: comparable_rental_2_gross_income_per_sqft | dtype: double ###
	Kmeans:
		counts: 0
		counts % = 0.00%
### Col: comparable_rental_2_boro_block_lot | dtype: string ###
### Col: comparable_renta

AnalysisException: "Union can only be performed on tables with the same number of columns, but the first table has 3 columns and the 2th table has 4 columns;;\n'Union\n:- AnalysisBarrier\n:     +- Project [rid#4311L, dist_c#4314, dist_c#4314]\n:        +- Filter ((dist_c#4314 < -38763.11363636366) || (dist_c#4314 > 54099.11363636366))\n:           +- Filter (c_no#4313L = cast(0 as bigint))\n:              +- LogicalRDD [rid#4311L, comparable_rental_3_gross_sqft#4312, c_no#4313L, dist_c#4314], false\n:- AnalysisBarrier\n:     +- Project [rid#4311L, dist_c#4314, dist_c#4314]\n:        +- Filter ((dist_c#4314 < -152226.8181818182) || (dist_c#4314 > 197226.8181818182))\n:           +- Filter (c_no#4313L = cast(1 as bigint))\n:              +- LogicalRDD [rid#4311L, comparable_rental_3_gross_sqft#4312, c_no#4313L, dist_c#4314], false\n+- AnalysisBarrier\n      +- Filter (c_no#4313L = cast(2 as bigint))\n         +- LogicalRDD [rid#4311L, comparable_rental_3_gross_sqft#4312, c_no#4313L, dist

In [ ]:
spark.stop()